## Model: distilgpt2

In [1]:
%pip uninstall -y pinecone-client
%pip install pinecone-client==3.0.0 --quiet

Found existing installation: pinecone-client 3.0.0Note: you may need to restart the kernel to use updated packages.

Uninstalling pinecone-client-3.0.0:
  Successfully uninstalled pinecone-client-3.0.0
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import pinecone
from dotenv import load_dotenv
import os
import torch
import logging

In [3]:
load_dotenv()
logging.basicConfig(level=logging.INFO)
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = "youtube-transcripts"

In [4]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


In [5]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(PINECONE_INDEX_NAME)

In [14]:
def retrieve_chunks(query, top_k=5):
    query_embedding = embedder.encode(query, show_progress_bar=False).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    chunks = []
    for match in results["matches"]:
        text = match["metadata"].get("text")
        if text:
            chunks.append(text)
    return chunks

In [15]:
def generate_response(query, chunks):
    context = "\n".join(chunks)
    prompt = f"Answer the question based on the context:\nContext: {context}\nQuestion: {query}"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [16]:
test_queries = [
    "What is ITSM in ServiceNow?",
    "Explain CMDB relationships.",
    "How does Incident Management work?"
    
]

In [17]:
for query in test_queries:
    try:
        chunks = retrieve_chunks(query)
        response = generate_response(query, chunks)
        print(f"Q: {query}\nA: {response}\n")
    except Exception as e:
        print(f"Error for query '{query}': {e}")

Token indices sequence length is longer than the specified maximum sequence length for this model (595 > 512). Running this sequence through the model will result in indexing errors


Q: What is ITSM in ServiceNow?
A: event management integration

Q: Explain CMDB relationships.
A: a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a.

Q: How does Incident Management work?
A: proactive versus reactive



## Results Evaluation

In [ ]:
PROJECT_ROOT = r"c:\Users\Mercy\OneDrive\Documents\ServiceNow classes\OneDrive\Project-Business-Case-Building-a-Multimodal-AI-ChatBot-for-YouTube-Video-QA"
RESULTS_DIR = os.path.join(PROJECT_ROOT, 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)
OUTPUT_FILE = os.path.join(RESULTS_DIR, 'model_outputs.csv')
print("Will save results to:", OUTPUT_FILE)

Will save results to: c:\Users\Mercy\OneDrive\Documents\ServiceNow classes\OneDrive\Project-Business-Case-Building-a-Multimodal-AI-ChatBot-for-YouTube-Video-QA\results\model_outputs.csv


In [ ]:
print("Current working directory:", os.getcwd())
print("Writing results to:", OUTPUT_FILE)

Current working directory: c:\Users\Mercy\OneDrive\Documents\ServiceNow classes\OneDrive\Project-Business-Case-Building-a-Multimodal-AI-ChatBot-for-YouTube-Video-QA\Sample Models
Writing results to: c:\Users\Mercy\OneDrive\Documents\ServiceNow classes\OneDrive\Project-Business-Case-Building-a-Multimodal-AI-ChatBot-for-YouTube-Video-QA\results\model_outputs.csv


In [ ]:
import csv
from datetime import datetime

def log_model_output(
    notebook,
    query,
    model_answer,
    output_file= OUTPUT_FILE,
    additional_fields=None
):
    # Compose data row
    timestamp = datetime.now().isoformat()
    row = {
        'timestamp': timestamp,
        'notebook': notebook,
        'query': query,
        'model_answer': model_answer
    }
    # Add additional fields if provided
    if additional_fields:
        row.update(additional_fields)
    # Write header if file does not exist
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    file_exists = os.path.isfile(output_file)
    with open(output_file, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=row.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)


## Metrics

In [45]:
questions = [
    "What is ITSM in ServiceNow?",
    "Explain CMDB relationships.",
    "How does Incident Management work?"
]

reference_answers = [
    "ITSM (IT Service Management) in ServiceNow refers to the strategic approach for designing, delivering, managing, and improving the way IT is used within an organization. ServiceNow ITSM offers a cloud-based platform to automate and streamline core IT processes such as Incident Management, Problem Management, Change Management, Request Fulfillment, and Knowledge Management. It enhances service quality, reduces operational costs, and improves user satisfaction through intelligent workflows, analytics, and a centralized service portal. ITSM in ServiceNow aligns IT services with business goals, ensuring efficient and consistent service delivery.",
    "CMDB (Configuration Management Database) relationships in ServiceNow define how Configuration Items (CIs) are connected and interact with each other. These relationships help build a dependency map that visually represents infrastructure, services, and their interconnections. Common relationship types include 'Depends on / Used by', 'Runs on / Hosted on', and 'Connected to'. For example, an application may run on a server, which in turn depends on a specific database. These relationships are essential for impact analysis, change planning, and root cause analysis, enabling IT teams to understand how changes or incidents affect related systems.",
    "Incident Management in ServiceNow focuses on restoring normal service operations as quickly as possible after an unplanned interruption. When an incident is reported—via the service portal, email, or phone—it is automatically logged and categorized. The system assigns priority based on impact and urgency, then routes the ticket to the appropriate support group. Technicians investigate, resolve the issue, and document the resolution. Users are kept informed through notifications. Once resolved, the incident is closed after verification. Incident Management helps minimize downtime, improves user satisfaction, and provides valuable data for trend analysis and continual service improvement."
]

In [46]:
model_answers = [
    "event management integration",
    "a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a. a.",
    "proactive versus reactive"
]

In [47]:
def exact_match(prediction, reference):
    """Returns 1 if the prediction matches the reference exactly (case-insensitive, stripped), else 0."""
    return int(prediction.strip().lower() == reference.strip().lower())

def f1_score(prediction, reference):
    """Compute token-level F1 score between prediction and reference."""
    pred_tokens = prediction.lower().split()
    ref_tokens = reference.lower().split()
    common = set(pred_tokens) & set(ref_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(ref_tokens)
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def rouge_l_score(prediction, reference):
    """Compute a simple ROUGE-L score based on longest common subsequence."""
    def lcs(X, Y):
        m = len(X)
        n = len(Y)
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        for i in range(m):
            for j in range(n):
                if X[i] == Y[j]:
                    dp[i + 1][j + 1] = dp[i][j] + 1
                else:
                    dp[i + 1][j + 1] = max(dp[i][j + 1], dp[i + 1][j])
        return dp[m][n]
    pred_tokens = prediction.lower().split()
    ref_tokens = reference.lower().split()
    lcs_len = lcs(pred_tokens, ref_tokens)
    if len(ref_tokens) == 0 or len(pred_tokens) == 0:
        return 0.0
    recall = lcs_len / len(ref_tokens)
    precision = lcs_len / len(pred_tokens)
    if recall + precision == 0:
        return 0.0
    return 2 * recall * precision / (recall + precision)

def bleu_score(prediction, reference):
    """Compute a simple BLEU-1 score (unigram precision)."""
    pred_tokens = prediction.lower().split()
    ref_tokens = reference.lower().split()
    if not pred_tokens or not ref_tokens:
        return 0.0
    matches = sum(1 for token in pred_tokens if token in ref_tokens)
    return matches / len(pred_tokens)

audio_filename = None  # or set to a placeholder string if needed

for question, ref_answer in zip(questions, reference_answers):
    # Retrieve relevant chunks for the question
    chunks = retrieve_chunks(question)
    answer = generate_response(question, chunks)
    em = exact_match(answer, ref_answer)
    f1 = f1_score(answer, ref_answer)
    rouge = rouge_l_score(answer, ref_answer)
    bleu = bleu_score(answer, ref_answer)
    # Remove confidence since generate_response does not return it
    log_model_output(
        notebook='3c_model_test_flat-t5-base',
        query=question,
        model_answer=answer,
        additional_fields={'em': em, 'f1': f1, 'rougeL': rouge, 'bleu': bleu}
    )